## 손글자 숫자 인식

0 ~ 9까지 숫자 이미지로 구성된 MNIST 데이터셋을 사용해 숫자를 분류할 것이다.  
따라서 출력층의 뉴런 수는 분류하고 싶은 클래스 수 10이된다.

In [1]:
import pickle
import numpy as np
from PIL import Image
from mnist import load_mnist
from functions import sigmoid, softmax

### Normalize, Pre-processing

mnist 데이터의 픽셀 값은 0 ~ 255 범위이다. 이를 0.0 ~ 1.0 범위로 변환하는 것을 ***정규화***.    
신경망의 입력 데이터에 특정 변환을 가하는 것을 ***전처리***.

아래 코드를 예로 들면 
>입력 이미지 데이터에 대한 전처리 작업으로 정규화를 수행하였다.

In [2]:
train_images, train_labels, test_images, test_labels = load_mnist(pickle_path="/Users/jun/Downloads/MNIST/mnist.pkl",
                                                                  flatten=True,
                                                                  normalize=False)

In [3]:
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

(60000, 784) (60000, 10)
(10000, 784) (10000, 10)


In [4]:
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

In [5]:
img = train_images[0]
label = train_labels[0]
print(label)

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [6]:
print(img.shape)
img = img.reshape(28, 28)
print(img.shape)

img_show(img)

(784,)
(28, 28)


### 신경망의 추론 처리
입력층 뉴런을 784개, 출력층 뉴런을 10개로 구성.  
은닉층은 총 두 개로 첫 번째 은닉층에는 50개의 뉴런, 두 번째 은닉층에는 100개의 뉴런을 배치할 것이다.(50과 100개는 임의)

In [7]:
def get_data():
    x_train, y_train, x_test, y_test = load_mnist(pickle_path="/Users/jun/Downloads/MNIST/mnist.pkl",
                                                      flatten=True,
                                                      normalize=True,
                                                      one_hot_label=False)

    return x_test, y_test

def init_network():
    with open("/Users/jun/Downloads/MNIST/sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
        
    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    
    return y

In [8]:
x, t = get_data()
network = init_network()

## 신경망 각 층의 배열 형상을 보니 행렬 곱 연산이 가능한 형태임을 알 수 있다.

### 이미지 1장을 입력했을 때 연산 흐름
<img src="img/deep_learning_images/fig_3-26.png" width=480 height=480>

### 이미지 100장을 입력했을 때 연산 흐름
<img src="img/deep_learning_images/fig_3-27.png" width=480 height=480>

In [20]:
W1, W2, W3 = network['W1'], network['W2'], network['W3']
b1, b2, b3 = network['b1'], network['b2'], network['b3']

print(x.shape)
print(W1.shape, W2.shape, W3.shape)
print(b1.shape, b2.shape, b3.shape)

(10000, 784)
(784, 50) (50, 100) (100, 10)
(50,) (100,) (10,)


In [19]:
accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y) # np.argmax 값이 최고로 높은 원소의 인덱스 값을 반환한다.
    
    if p == t[i]:
        accuracy_cnt += 1
        
print(f"Accuracy : {str(float(accuracy_cnt) / len(x))}")

Accuracy : 0.9352


## 배치 처리 적용

배치 처리는 이미지 1장당 처리 시간을 대폭 줄여준다.  
1. 수치 계산 라이브러리 대부분이 큰 배열을 효율적으로 처리할 수 있도록 최적회되어 있음
2. 커다란 신경망에서는 데이터 전송이 병목으로 작용하는 경우가 있는데, 배치 처리를 함으로써 버스에 주는 부하를 줄인다.

즉, 배치 처리를 수행함으로써, 큰 배열로 이루어진 계산을 하게 되고, 컴퓨터에서는 큰 배열을 한꺼번에 계산하는 것이 분할된 작은 배열을 여러 번 계산하는 것보다 빠르다.

In [26]:
batch_size = 100
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i : i + batch_size]
    y_batch = predict(network, x_batch)
    
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i : i + batch_size])
    
print(f"Accuracy : {str(float(accuracy_cnt) / len(x))}")

Accuracy : 0.9352


In [30]:
sample_y = np.array([1, 2, 1, 0])
sample_t = np.array([1, 2, 0, 0])

print(sample_y == sample_t)
print(np.sum(sample_t == sample_y))

[ True  True False  True]
3
